In [295]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import os
os.chdir('/Users/shannontran/Downloads') #changing directory to where my files are

In [296]:
# Connecting to Mongo db

In [297]:
client = MongoClient('localhost', 27017)

In [298]:
db = client.proj2_mongo

In [299]:
# Importing datasets

In [300]:
# students.csv
students_df = pd.read_csv('students.csv')
students_df = students_df.iloc[1:] #removes first row with datatypes
students_df.head()

students_df = students_df.astype({'student_id':int, 'total_credits':int}) #changing data types from str to int


In [301]:
# in clubs 
in_club_df = pd.read_csv('in_club.csv')[1:]
in_club_df = in_club_df.astype({'student_id':int, 'club_id':int})

In [302]:
# clubs
club_df = pd.read_csv('club.csv')[1:]
club_df = club_df.astype({'club_id':int, 'num_members':int})

In [303]:
# Converting to dict
students_dict = students_df.to_dict('records')
in_club_dict = in_club_df.to_dict('records')
club_dict = clubs_df.to_dict('records')


In [304]:
# Nests club into in_club
for i in range(0, len(in_club_dict)):
    club_id = in_club_dict[i]['club_id']
    club_info = club_df[club_df['club_id'] == club_id][['club_name']].iloc[0].to_dict()
    in_club_dict[i]['club_id'] = club_info
    #print(club_info)

In [305]:
# Nests student into in_club
for i in range(0, len(in_club_dict)):
    student_id = in_club_dict[i]['student_id']
    student_info = students_df[students_df['student_id'] == student_id][['first_name', 'last_name']].iloc[0].to_dict()
    in_club_dict[i]['student_id'] = student_info


In [306]:
in_club_dict[1]

{'student_id': {'first_name': 'Forge', 'last_name': 'Launch'},
 'club_id': {'club_name': 'Club Running'},
 'role': 'Member'}

In [307]:
# Connecting to MongoDB

In [308]:
in_club_collection = db.in_club # create collection

In [318]:
in_club_collection.insert_many(in_club_dict) #inserts nested dictionary into in_club collection

In [328]:
# Who are club presidents?
query1 = in_club_collection.find({'role':'President'}, {'_id':0}) # fields set to zero are removed from output (for cleaner look)

In [329]:
import pprint #pretty print
for e in query1:
    pprint.pprint(e)

{'club_id': {'club_name': 'Club Running'},
 'role': 'President',
 'student_id': {'first_name': 'Mary', 'last_name': 'Jane'}}
{'club_id': {'club_name': 'The Math Wizards'},
 'role': 'President',
 'student_id': {'first_name': 'Jon', 'last_name': 'Kashmeeri'}}
{'club_id': {'club_name': 'Computational Data Sciences'},
 'role': 'President',
 'student_id': {'first_name': 'Kennedy', 'last_name': 'Vorkin'}}


In [327]:
query2 = in_club_collection.find({'role':'Member'}, {'_id':0}) # fields set to zero are removed from output (for cleaner look)

for e in query2:
    pprint.pprint(e)

{'club_id': {'club_name': 'Club Running'},
 'role': 'Member',
 'student_id': {'first_name': 'Forge', 'last_name': 'Launch'}}
{'club_id': {'club_name': 'Club Running'},
 'role': 'Member',
 'student_id': {'first_name': 'Jasmine', 'last_name': 'Wang'}}
{'club_id': {'club_name': 'Alternative Spring Break'},
 'role': 'Member',
 'student_id': {'first_name': 'John', 'last_name': 'Doe'}}
{'club_id': {'club_name': 'The Math Wizards'},
 'role': 'Member',
 'student_id': {'first_name': 'Forge', 'last_name': 'Launch'}}
{'club_id': {'club_name': 'The Math Wizards'},
 'role': 'Member',
 'student_id': {'first_name': 'John', 'last_name': 'Doe'}}
{'club_id': {'club_name': 'The Math Wizards'},
 'role': 'Member',
 'student_id': {'first_name': 'Jose', 'last_name': 'Pein'}}
{'club_id': {'club_name': 'Madison House'},
 'role': 'Member',
 'student_id': {'first_name': 'Mark', 'last_name': 'Maguire'}}
{'club_id': {'club_name': 'Madison House'},
 'role': 'Member',
 'student_id': {'first_name': 'Silas', 'last_nam

In [325]:
# Who are seniors by credits? (>=90 credits)
students_collection.insert_many(students_dict)

In [326]:
query3 = students_collection.find({'total_credits':{'$gte':90}}, {'living_status':0, 'student_id':0, 'dept_code':0}) # fields set to zero are removed from output (for cleaner look)

for e in query3:
    pprint.pprint(e)

{'_id': ObjectId('6297ca7257c9c4be73ede123'),
 'first_name': 'John',
 'last_name': 'Doe',
 'student_year': 'Senior',
 'total_credits': 100}
{'_id': ObjectId('6297ca7257c9c4be73ede124'),
 'first_name': 'Jon',
 'last_name': 'Kashmeeri',
 'student_year': 'Senior',
 'total_credits': 113}
{'_id': ObjectId('6297ca7257c9c4be73ede126'),
 'first_name': 'Mark',
 'last_name': 'Maguire',
 'student_year': 'Senior',
 'total_credits': 95}
{'_id': ObjectId('6297ca7257c9c4be73ede12d'),
 'first_name': 'Silas',
 'last_name': 'French',
 'student_year': 'Senior',
 'total_credits': 90}
{'_id': ObjectId('6297ca7257c9c4be73ede12e'),
 'first_name': 'Ronald',
 'last_name': 'McDonald',
 'student_year': 'Senior',
 'total_credits': 102}
{'_id': ObjectId('6297ca7257c9c4be73ede12f'),
 'first_name': 'Walt',
 'last_name': 'White',
 'student_year': 'Senior',
 'total_credits': 120}
{'_id': ObjectId('6297ca7257c9c4be73ede132'),
 'first_name': 'Bo',
 'last_name': 'Bo',
 'student_year': 'Senior',
 'total_credits': 92}
{'_i

In [324]:
#students_collection.delete_many({})

In [317]:
#in_club_collection.delete_many({})